In [ ]:
import pandas as pd 
import pandas_ta as ta 


In [ ]:
import pandas as pd 
from datetime import date, datetime
import yfinance as yf

tickers = ['ADBE', 'GE', 'INTU', 'MS', 'VRTX', 'XOM']

dataframes = []

overall_df = pd.DataFrame() 

count1 = 0
for ticker in tickers:
    
    data_df = pd.read_csv(f'{ticker.lower()}/data/test/OVERALL_prediction_by_model.csv')
    data_df['Date'] = pd.to_datetime(data_df['Date'])
    data_df['Close'] = data_df['close']
    data_df = data_df[['Date', 'Close']]
    data_df = data_df.set_index('Date')  
       
    dataframes.append(data_df)
    
    if len(overall_df) == 0: 
        overall_df = data_df[['Close']]
        overall_df = overall_df.rename(columns = {'Close': f'{ticker}'})
    else: 
        overall_df = pd.concat([overall_df, data_df['Close']], axis = 1)
        overall_df = overall_df.rename(columns = {'Close': f'{ticker}'})
store = overall_df.copy()

In [ ]:
overall_df

In [ ]:
        
initial_sum = 10000000 * 0.75

overall_df = overall_df.reset_index(drop = False)

one_share = initial_sum / 6

num_shares = {} 

for item in overall_df.columns: 
    if item == 'Date': continue 
    num_shares[item] = one_share/overall_df[item][0]

    
adbe_shares = num_shares['ADBE']
ge_shares = num_shares['GE']
intu_shares = num_shares['INTU']
ms_shares = num_shares['MS']
vrtx_shares = num_shares['VRTX']
xom_shares = num_shares['XOM']

adbe_initial = overall_df['ADBE'][0].item()
ge_initial = overall_df['GE'][0].item()
intu_initial = overall_df['INTU'][0].item()
ms_initial = overall_df['MS'][0].item()
vrtx_initial = overall_df['VRTX'][0].item()
xom_initial = overall_df['XOM'][0].item()

average_returns = overall_df.copy()[['Date']]

for column in overall_df.columns: 
    if column == 'Date': continue 
    
    if column == 'ADBE': tools = adbe_shares
    elif column == 'GE': tools = ge_shares
    elif column == 'INTU': tools = intu_shares
    elif column == 'MS': tools = ms_shares
    elif column == 'VRTX': tools = vrtx_shares
    elif column == 'XOM': tools = xom_shares 
        
    average_returns[column] = overall_df.apply(lambda x: x[column]*tools, axis = 1)
    
average_returns['average_returns'] = average_returns.apply(lambda x: round((x['ADBE'] + x['GE'] + x['INTU'] + x['MS'] + x['VRTX'] + x['XOM'] - initial_sum)/initial_sum, 4), axis = 1)
average_returns = average_returns[['Date', 'average_returns']]
average_returns = average_returns.set_index('Date')


In [ ]:

market = yf.download('^gspc', start="2019-01-01", end="2024-01-01", auto_adjust=True)
initial = market['Close'][0]
market['market_returns'] = market.apply(lambda x: (x['Close'] - initial) / initial, axis = 1)
market_returns = market[['market_returns']]

baseline = pd.concat([market_returns, average_returns], axis = 1)
baseline 

portfolio1 = pd.read_csv('ammended_portfolio_outcome/forecasted/a2c_portfolio.csv')
portfolio1['portfolio_returns'] = portfolio1.apply(lambda x: x['daily_return'] - 1, axis = 1)
portfolio1['Date'] = pd.to_datetime(portfolio1['date'])
portfolio1 = portfolio1.set_index('Date')
portfolio = portfolio1[['portfolio_returns']]
portfolio

combined = pd.concat([portfolio, baseline], axis = 1)
combined

## dividned returns

In [ ]:
tickers = ['MMM', 'PEP', 'PFE', 'UPS', 'VZ']

dataframes_DIV = []

overall_df = pd.DataFrame() 

count1 = 0
for ticker in tickers:
    
    data_df = yf.download(ticker, start="2019-01-01", end="2024-01-01", auto_adjust=True)
    dataframes_DIV.append(data_df)
    
    if len(overall_df) == 0: 
        overall_df = data_df[['Close']]
        overall_df = overall_df.rename(columns = {'Close': f'{ticker}'})
    else: 
        overall_df = pd.concat([overall_df, data_df['Close']], axis = 1)
        overall_df = overall_df.rename(columns = {'Close': f'{ticker}'})

        
        
        
initial_sum = 10000000 * 0.25

overall_df = overall_df.reset_index(drop = False)

one_share = initial_sum / 5

num_shares_div = {} 

for item in overall_df.columns: 
    if item == 'Date': continue 
    num_shares_div[item] = one_share/overall_df[item][0]        
        
        
        
mmm_shares = num_shares_div['MMM']
pep_shares = num_shares_div['PEP']
pfe_shares = num_shares_div['PFE']
ups_shares = num_shares_div['UPS']
vz_shares = num_shares_div['VZ']

mmm_initial = overall_df['MMM'][0].item()
pep_initial = overall_df['PEP'][0].item()
pfe_initial = overall_df['PFE'][0].item()
ups_initial = overall_df['UPS'][0].item()
vz_initial = overall_df['VZ'][0].item()

average_returns = overall_df.copy()[['Date']]

for column in overall_df.columns: 
    if column == 'Date': continue 
    
    if column == 'MMM': tools = mmm_shares
    elif column == 'PEP': tools = pep_shares
    elif column == 'PFE': tools = pfe_shares
    elif column == 'UPS': tools = ups_shares
    elif column == 'VZ': tools = vz_shares
    
        
    average_returns[column] = overall_df.apply(lambda x: x[column]*tools, axis = 1)
    
average_returns['dividend_average_returns'] = average_returns.apply(lambda x: round((x['MMM'] + x['PEP'] + x['PFE'] + x['UPS'] + x['VZ'] - initial_sum)/initial_sum, 4), axis = 1)
average_returns = average_returns[['Date', 'dividend_average_returns']]
average_returns = average_returns.set_index('Date')


In [ ]:
combined1 = pd.concat([combined, average_returns], axis = 1)
combined1

In [ ]:
num_shares_div

In [ ]:
from tqdm.auto import tqdm

shortlisted = ['MMM', 'PEP', 'PFE', 'UPS', 'VZ']

overall = {}
train = {} 
test = {} 

for ticker in tqdm(shortlisted): 

    df = pd.DataFrame() 
    df = df.ta.ticker(ticker) 
    df = df.reset_index(drop = False) 
    df['Date'] = df.apply(lambda x: x.Date.date(), axis = 1)
    df['dividend_rate'] = df.apply(lambda x: 0 if x['Dividends'] == 0 else x['Dividends']/x['Close'], axis = 1)
    dividend = df.copy()[df['Date'] >= date(2014, 1, 1)][df['Date'] < date(2024, 1, 1)]
    dividend_train = dividend.copy()[dividend['Date'] >= date(2014, 1, 1)][dividend['Date'] < date(2019, 1, 1)]
    dividend_test = dividend.copy()[dividend['Date'] >= date(2019, 1, 1)][dividend['Date'] < date(2024, 1, 1)]
        
    overall[ticker] = dividend 
    train[ticker] = dividend_train
    test[ticker] = dividend_test


In [ ]:
num_shares_div

In [ ]:


for item in test: 
    
    shares = num_shares_div[item]
    
    print(f'{item} - {shares}')
    
    reference = test[item]
    
    reference['dividend_amount'] = reference.apply(lambda x: x['Dividends']* shares, axis = 1)
    test[item] = reference
    
    


In [ ]:

consol_df = test['VZ'][['Date']]
consol_df = consol_df.set_index('Date')

for item in test:
    
    temp = test[item].set_index('Date')[['dividend_amount']]
    
    consol_df = pd.concat([consol_df, temp], axis = 1)
    
    consol_df = consol_df.rename(columns = {'dividend_amount': item})
    
consol_df['total'] = consol_df.apply(lambda x: x['MMM'] + x['PEP'] + x['PFE'] + x['UPS'] + x['VZ'], axis = 1)
consol_df    

In [ ]:
dividend = consol_df['total']

value = [] 
initial_cyclic_sum = 10000000 * 0.75


for index in range(len(dividend)): 
    if index == 0: 
        instance = initial_cyclic_sum
        value.append(instance)
    else: 
        instance = value[index - 1] + dividend[index]*(1 - 0.00125)
        value.append(instance)
        
consol_df['cyclic_value'] = value
consol_df['return'] = combined['portfolio_returns']
consol_df['return_amount'] = consol_df.apply(lambda x: x['cyclic_value']*(x['return'] + 1), axis = 1)
consol_df['resultant_return'] = consol_df.apply(lambda x: (x['return_amount'] - initial_cyclic_sum)/initial_cyclic_sum, axis = 1)
consol_df

In [ ]:
total = sum(consol_df['total'])
print(f'Dividend Received: {total}')

In [ ]:
combined1['dividend_adjusted_return'] = consol_df['resultant_return']
combined1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

target = combined1.copy() 

fig, ax1 = plt.subplots(figsize=(14, 8))
fig.suptitle('Returns Comparison', fontsize=14)

ax1.plot(target.index, target['portfolio_returns'], '-r', label = 'Cyclic Portfolio Returns (A2C)')
ax1.plot(target.index, target['average_returns'], '-g', label = 'Cyclic Portfolio Returns (Equal Weights)')

ax1.plot(target.index, target['market_returns'], '-b', label = 'Benchmark Market Returns (S&P500)')

ax1.plot(target.index, target['dividend_average_returns'], '-y', label = 'Dividend Portfolio Returns (Equal Weights, Stock Price Only)')
ax1.plot(target.index, target['dividend_adjusted_return'], 'black', label = 'Cyclic Portfolio Returns with Dividend Injection (A2C)')

ax1.set_ylabel("Returns")
ax1.set_xlabel("Date")
plt.grid()
fig.legend(loc = 'upper left')

<b> Statistics </b>

Dividend Average Portfolio

In [ ]:
div_final_value = 2500000*(1+combined1['dividend_average_returns'][-1])
div_final_value

In [ ]:
combined2 = combined1.copy()

In [ ]:
dividend_initial = 10000000 * 0.25
combined2['dividend_portfolio'] = combined2.apply(lambda x: (1+x['dividend_average_returns'])* dividend_initial, axis = 1)
combined2

Cyclic Portfolio With Injections

In [ ]:
consol_df['return_amount'][-1]

Cyclic Portfolio without Injections

In [ ]:
combined2

In [ ]:
(combined2['portfolio_returns'][-1]+1)*(10000000*0.75)

If dividend tagged to S&P500

In [ ]:
trial = consol_df[['total']]
trial

In [ ]:
# get snp data 
ticker = '^gspc'
market = pd.DataFrame() 
market = market.ta.ticker(ticker) 
market = market.reset_index(drop = False) 
market['Date'] = market.apply(lambda x: x.Date.date(), axis = 1)

market = market[['Date', 'Close']]

end_val = market[market['Date'] < date(2024, 1, 1)]
end_val = end_val['Close'][-1: ].item()

market['train_close'] = end_val 
market['market_returns'] = market.apply(lambda x: (x['train_close'] - x['Close'])/x['Close'] + 1, axis = 1)

market = market[['Date', 'market_returns']]

market

In [ ]:
trial = pd.merge(trial, market, how = 'left', on='Date')
trial['adj_returns'] = trial.apply(lambda x: x['total'] * x['market_returns'], axis = 1)
trial

In [ ]:
sum(trial['adj_returns'])

Dividend portfolio

In [ ]:
div_final_value = 2500000*(1+combined1['dividend_average_returns'][-1]) + sum(trial['adj_returns'])

In [ ]:
div_final_value

Dividend Received

In [ ]:
total = sum(consol_df['total'])
print(f'Dividend Received: {total}')